# IS-NMF

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git
pip install soundfile

In [ ]:
%cd "/content/audio_source_separation/egs/nmf-example/is-nmf"

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from algorithm.nmf import ISNMF

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
fft_size, hop_size = 1024, 256

In [ ]:
x, sr = sf.read("../../../dataset/sample-song/sample_electric-guitar_8000-1.wav")

### Original Source

In [ ]:
ipd.Audio(x, rate=sr)

In [ ]:
_, _, spectrogram = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)
X = np.abs(spectrogram)**2

In [ ]:
n_bases = 6

### Execution by NMF

In [ ]:
np.random.seed(111)
nmf = ISNMF(n_bases=n_bases)

In [ ]:
nmf.update(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.show()

### Recostructed Signal

In [ ]:
X[X < 1e-12] = 1e-12

In [ ]:
Z = nmf.base @ nmf.activation

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

### Factorized Signals

In [ ]:
for idx in range(n_bases):
    Z = nmf.base[:, idx: idx+1] * nmf.activation[idx: idx+1, :]

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_bases):
    estimated_spectrogram = nmf.base[:, idx: idx + 1] @ nmf.activation[idx: idx + 1, :]

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()

## Domain Parameter
The cost function of IS-NMF is defined as
\begin{align}
    \mathcal{L}
    = \sum_{i,j}\left(\log\frac{d_{ij}}{\displaystyle\left(\sum_{k}t_{ik}v_{kj}\right)^{\frac{2}{p}}} - \frac{d_{ij}}{\displaystyle\left(\sum_{k}t_{ik}v_{kj}\right)^{\frac{2}{p}}} - 1\right),
\end{align}
where $d_{ij}$ is target spectrogram, $p$ is domain parameter.
By default $p=2$.

In [ ]:
domain = 1

In [ ]:
np.random.seed(111)
nmf = ISNMF(n_bases=n_bases, domain=domain)

In [ ]:
nmf.update(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.show()

In [ ]:
Z = (nmf.base @ nmf.activation)**(2 / domain)

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

In [ ]:
for idx in range(n_bases):
    Z = (nmf.base[:, idx: idx+1] * nmf.activation[idx: idx+1, :])**(2 / domain)

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_bases):
    estimated_spectrogram = (nmf.base[:, idx: idx + 1] @ nmf.activation[idx: idx + 1, :])**(2 / domain)

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()